In [1]:
import json
import numpy as np

json_dir='/Users/tommy/workspace/data/psout/ezio_checker.json'

class confmat_stat(object):
    def __init__(self,path):
        try:
            with open(path, 'rb')  as f:
                self.data = json.load(f)
                #print "read data complete"
                self.enable = True
        except Exception as e:
                print e
        self.order = self.data.keys()
        self.class_good = []
        self.class_bad = []
        
    def category(self):
        for orderId in self.order:
            viewList = self.data[orderId].keys()
            for viewId in viewList:
                one_view = self.data[orderId][viewId]
                if (one_view['label'] == 'good'):
                    self.class_good.append((one_view,(orderId,viewId)))
                elif (one_view['label'] == 'bad'):
                    self.class_bad.append((one_view,(orderId,viewId)))
        print('class good total: ',len(self.class_good))
        print('class bad total: ',len(self.class_bad))
        
    def flatten_data(self,one_view):
        #print(one_view.keys())
        recall_vec = np.array(one_view['recall']).flatten()
        iou_vec = np.array(one_view['iou']).flatten()
        confmat = np.array(one_view['matrix']).flatten()
        prec_vec = np.array(one_view['precision']).flatten()
        
        feature_vec = np.array([])
        feature_vec = np.hstack((feature_vec,confmat))
        feature_vec = np.hstack((feature_vec,prec_vec))
        feature_vec = np.hstack((feature_vec,recall_vec))
        feature_vec = np.hstack((feature_vec,iou_vec))
        
        return feature_vec
    
    def concat(self):
        data = np.array([])
        label = []
        id_list = []
        dim = 0
        for class_good_, id_tuple in self.class_good:
            data = np.hstack((data,self.flatten_data(class_good_)))
            dim = len(self.flatten_data(class_good_))
            label.append(1)
            id_list.append(id_tuple)
        for class_bad_, id_tuple in self.class_bad:
            data = np.hstack((data,self.flatten_data(class_bad_)))
            label.append(2)
            id_list.append(id_tuple)
        #print(dim)
        data = np.reshape(data, (-1,dim))
        label = np.array(label)
        self.id_list = id_list
        return data,label,id_list
    
    def get_label(self,id_tuple):
        orderId,viewId = id_tuple
        one_view = self.data[orderId][viewId]
        return one_view['label']
    
stat = confmat_stat(json_dir)
stat.category()
data,label,id_list = stat.concat()
print(data.shape,label.shape,len(id_list))
print(stat.get_label(id_list[0]))

('class good total: ', 74)
('class bad total: ', 77)
((151, 18), (151,), 151)
good


In [14]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
X = data
y = label

print(X.shape, y.shape)
clf = LinearDiscriminantAnalysis(store_covariance=False,n_components=1)
clf.fit(X, y)
print("test acc",clf.score(X,y))

print("coef",clf.coef_)
print("interc",clf.intercept_)
print("scaling",clf.scalings_.T)

scaling = clf.scalings_.T
coef = np.array(clf.coef_)
inter = clf.intercept_

print(data[0])

for idx,test_data in enumerate(data):
    test_data = np.asmatrix(test_data)

    inference = np.dot(test_data,coef.T) + inter
    inference_prob = np.reciprocal(np.exp(-inference) + 1)
    
    raw_prediction = clf.predict(test_data)

    if(raw_prediction == 2):
        prediction = 'bad'
    elif (raw_prediction == 1):
        prediction = 'good'
    ground_truth = stat.get_label(id_list[idx])
    if(prediction != ground_truth):
        print("class pred",prediction,"ground truth",ground_truth)
        
        print(clf.predict_proba(test_data))
        
        print("ID",id_list[idx])
        
    #print("transform",clf.decision_function(test_data))
#print(clf.covariance_)
#print(clf.explained_variance_ratio_)

((151, 18), (151,))
('test acc', 0.8145695364238411)
('coef', array([[ 1.91421674e-04, -1.43421627e-03,  4.60424363e-05,
         1.04678782e-03,  9.96335743e-05,  1.00164328e-03,
         4.08150387e-03, -8.88615918e-04, -2.49910258e-04,
         2.90427129e+01, -1.66012712e+01, -2.05925450e+00,
        -3.62422893e+00,  2.70976973e+01, -2.35328940e+00,
        -2.97229115e+01, -1.47014008e+01,  3.36642418e+00]]))
('interc', array([-0.67758561]))
('scaling', array([[ 1.13885171e-04, -8.53278326e-04,  2.73926699e-05,
         6.22780100e-04,  5.92763944e-05,  5.95921633e-04,
         2.42826612e-03, -5.28676683e-04, -1.48682601e-04,
         1.72787869e+01, -9.87682621e+00, -1.22514105e+00,
        -2.15621315e+00,  1.61216116e+01, -1.40007534e+00,
        -1.76834670e+01, -8.74650980e+00,  2.00283377e+00]]))
[2.04540000e+04 7.47300000e+03 0.00000000e+00 3.44600000e+03
 1.87660000e+04 4.00000000e+00 1.00000000e+00 3.20000000e+01
 0.00000000e+00 8.55780093e-01 7.14323779e-01 1.00000000e

In [3]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(X, y)
print(qda.score(X,y))

0.8079470198675497


/Users/tommy/anaconda2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
